# NBA Player Data Analysis Project

## Project Outline

1. **Introduction**
   - Overview of the project
   - Goals and objectives

2. **Data Collection**
   - Import necessary libraries
   - Fetch player data from the NBA API

3. **Data Processing**
   - Load data into a Pandas DataFrame
   - Data cleaning and transformation

4. **Data Analysis**
   - Exploratory data analysis (EDA)
   - Visualizations

5. **Conclusion**
   - Summary of findings
   - Future work


## 0. Environment

### Python

This project is written in Python, which means that Python must be installed in your environment to run the project. The minimum supported version is 3.10.

#### Windows

You can use the Windows package manager `winget`, or the [installer](https://www.python.org/downloads/windows/) from the website.
```powershell
# you can change the version in the package name to your desired version
winget install Python.Python.3.12
```

#### MacOS
Python is already installed by default on recent versions of MacOS. If you have an older version that is not supported, you can use the [Homebrew](https://brew.sh/) package manager to install it, or the [installer](https://www.python.org/downloads/macos/) from the website.
```zsh
brew install python
```

#### Linux
Python is already installed by default on most distributions of Linux. If it isn't, you can use your distribution's package manager to install Python.

### Dependencies

This project uses [uv](https://github.com/astral-sh/uv) to manage its dependencies. You can install the dependencies with the `uv` command:

`uv add pandas`

If you don't want to use `uv`, a `requirements.txt` is also provided. You can install this using `pip`:

`pip install -r requirements.txt`

### Imports

In [ ]:
import os
import time

import pandas as pd

### Environment Variables

We will load all our environment variables from a `.env` file, if one is provided.

If database information is provided, all dataframes used for analysis are uploaded to it. We use [Microsoft SQL Server](https://www.microsoft.com/en-us/sql-server/sql-server-downloads) by default but any kind of database is supported.

In [ ]:
from dotenv import load_dotenv

load_dotenv()
DB_TYPE = os.getenv("DB_TYPE", "sqlserver")
DB_USER = os.getenv("DB_USER", "sqladmin")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "1433")
DB_NAME = os.getenv("DB_NAME", "dataframes")
DB_DRIVER = os.getenv("DB_DRIVER")  # some databases require a database driver

### Presentation

By default, Pandas dataframes are truncated when they are printed. We want to be able to view all of the data at once, so we embed the dataframe in a scrollable element.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML, display


def custom_scrollable_display(df: pd.DataFrame, max_height=400):
    """
    Custom display function to render DataFrames as scrollable elements.

    Parameters:
    - df: The DataFrame to display.
    - max_height: The maximum height of the scrollable area in pixels.
    """
    style = f"""
    <style>
    .scrollable-dataframe {{
        display: inline-block;
        white-space: nowrap;
        overflow-x: scroll;
        max-height: {max_height}px;
        overflow-y: scroll;
    }}
    </style>
    """
    display(HTML(style + f'<div class="scrollable-dataframe">{df.to_html()}</div>'))


def custom_display_hook(df):
    custom_scrollable_display(df)
    return ""


# hook up the custom display function to the automatic printer
InteractiveShell.instance().display_formatter.formatters["text/html"].for_type(
    pd.DataFrame, custom_display_hook
);

### Pre-Commit Hooks (Developer Only)

This notebook uses `nbstripout` to strip notebook output from Git commits. If you are committing code, please run the following command to set up the Git filter.

`uv` is required for the pre-commit hooks, so make sure it is installed before you commit code. These hooks will keep the `requirements.txt` file updated.

In [ ]:
!nbstripout --install
!pre-commit install

## 1. Data Collection

### Fetch Player Data from NBA API

`nba_api` provides static player and team information, which we will download here so that we can reuse it without requesting the API unnecessarily.

In [ ]:
from nba_api.stats.static import players, teams

DATA_DIR = "../../data"

PLAYERS_LIST_FILE = "players_list.csv"
TEAMS_LIST_FILE = "teams_list.csv"

if os.path.exists(f"{DATA_DIR}/{PLAYERS_LIST_FILE}"):
    players_list = pd.read_csv(f"{DATA_DIR}/{PLAYERS_LIST_FILE}")
else:
    players_list = pd.DataFrame(players.get_players())
    players_list.to_csv(f"{DATA_DIR}/{PLAYERS_LIST_FILE}")

if os.path.exists(f"{DATA_DIR}/{TEAMS_LIST_FILE}"):
    teams_list = pd.read_csv(f"{DATA_DIR}/{TEAMS_LIST_FILE}")
else:
    teams_list = pd.DataFrame(teams.get_teams())
    teams_list.to_csv(f"{DATA_DIR}/{TEAMS_LIST_FILE}")

### Fetch Game Data



We're only interested in games that are either in the regular season or in the playoffs. We'll add an enum to distinguish the type of game and use it to differentiate them.

In [ ]:
from enum import Enum


class SeasonType(Enum):
    PRESEASON = 1
    REGULAR_SEASON = 2
    ALL_STAR = 3
    PLAYOFFS = 4
    PLAY_IN = 5
    NBA_CUP = 6


class Season:
    def __init__(self, season_id: int) -> None:
        season_id_str = str(season_id)
        self.season_type = SeasonType(int(season_id_str[0]))
        self.season_year = int(season_id_str[1:])

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import LeagueIDNullable

START_SEASON = 2023
END_SEASON = 2024
GAMES_LIST_FILE = "games_list.csv"
if os.path.exists(f"{DATA_DIR}/{GAMES_LIST_FILE}"):
    games_list: pd.DataFrame = pd.read_csv(f"{DATA_DIR}/{GAMES_LIST_FILE}")
else:
    games_list = pd.DataFrame()
    for season in range(START_SEASON, END_SEASON + 1):
        # put season into the correct form e.g. 2023 -> 2023-24
        season_str = f"{season}-{str(season + 1)[2:]}"
        print(f"Fetching games for season: {season_str}", end="\r")
        gamefinder = leaguegamefinder.LeagueGameFinder(
            season_nullable=season_str, league_id_nullable=LeagueIDNullable.nba
        )
        games = gamefinder.get_data_frames()[0]
        games_list = pd.concat([games_list, games], ignore_index=True)
        time.sleep(0.6)
    games_list.to_csv(f"{DATA_DIR}/{GAMES_LIST_FILE}", index=False)
# games_list["SEASON_ID"].unique()
# games_list.loc[(games_list["TEAM_NAME"] == "San Antonio Spurs") & (games_list["SEASON_ID"] == 22023)]
games_list.head()

### Fetch Play by Plays

In [ ]:
from nba_api.stats.endpoints import playbyplayv3
from requests.exceptions import ReadTimeout

# took 483 minutes to download up to 2012
PBP_LIST_FILE = "../../data/pbp_list.csv"
if os.path.exists(PBP_LIST_FILE):
    pbp_list = pd.read_csv(PBP_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    pbp_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if (
            season.season_type != SeasonType.REGULAR_SEASON
            and season.season_type != SeasonType.PLAYOFFS
        ):
            continue
        print(f"Fetching play by play for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                pbpfinder = playbyplayv3.PlayByPlayV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../../data/err.log", "a") as f:
                    print(f"{game_id} does not have a play by play", file=f)
                err = True
                break
        if err:
            continue
        pbp = pbpfinder.get_data_frames()[0]
        pbp_list = pd.concat([pbp_list, pbp], ignore_index=True)
        time.sleep(0.6)
    pbp_list.to_csv(PBP_LIST_FILE, index=False)
pbp_list.head()

### Fetch Box Scores

#### Player Track

In [ ]:
from nba_api.stats.endpoints import boxscoreplayertrackv3
from requests.exceptions import ReadTimeout

# took 483 minutes to download up to 2012
BOXSCORE_PT_LIST_FILE = "../../data/boxscore_pt_list.csv"
if os.path.exists(BOXSCORE_PT_LIST_FILE):
    boxscore_pt_list = pd.read_csv(BOXSCORE_PT_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    boxscore_pt_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if (
            season.season_type != SeasonType.REGULAR_SEASON
            and season.season_type != SeasonType.PLAYOFFS
        ):
            continue
        print(f"Fetching box score for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                boxscorefinder = boxscoreplayertrackv3.BoxScorePlayerTrackV3(
                    f"{game_id:010}"
                )
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a player track box score", file=f)
                err = True
                break
        if err:
            continue
        boxscore_pt = boxscorefinder.get_data_frames()[0]
        boxscore_pt_list = pd.concat([boxscore_pt_list, boxscore_pt], ignore_index=True)
        time.sleep(0.6)
    boxscore_pt_list.to_csv(BOXSCORE_PT_LIST_FILE, index=False)
boxscore_pt_list.head()

## 2. Data Processing

### Recency

Our analysis will consider only the last two years, so we'll get rid of data from before that.

In [ ]:
season_year = games_list["SEASON_ID"].astype(str).str[1:].astype(int)
games_list["season_year"] = season_year
current_season_year = 2023  # Replace with the current season's start year
games_list = games_list[
    games_list["season_year"].isin([current_season_year, current_season_year - 1])
]

In [ ]:
pbp_list = pbp_list[pbp_list["gameId"].isin(games_list["GAME_ID"].unique())]

In [ ]:
boxscore_pt_list = boxscore_pt_list[
    boxscore_pt_list["gameId"].isin(games_list["GAME_ID"].unique())
]

### Unnecessary Columns

Some of this data isn't useful to us, so we'll drop it to ignore the noise.

In [ ]:
pbp_columns_to_drop = [
    "actionNumber",
    "pointsTotal",
    "videoAvailable",
    "actionId",
    "playerNameI",
    "teamTricode",
]
pbp_list.drop(
    columns=[col for col in pbp_columns_to_drop if col in pbp_list.columns],
    inplace=True,
)
pbp_list.head()

In [ ]:
bspt_columns_to_drop = [
    "teamCity",
    "teamName",
    "teamTricode",
    "teamSlug",
    "playerNameI",
    "teamTricode",
    "playerSlug",
    "jerseyNum",
]
boxscore_pt_list.drop(
    columns=[col for col in bspt_columns_to_drop if col in boxscore_pt_list.columns],
    inplace=True,
)
boxscore_pt_list.head()

### Categorization

To save on memory, we will also turn variables that can be understood as categorical variables into that type.

In [ ]:
pbp_categorical_columns = [
    "gameId",
    "teamId",
    "shotResult",
    "isFieldGoal",
    "location",
    "actionType",
    "subType",
    "personId",
    "playerName",
]
pbp_list[pbp_categorical_columns] = pbp_list[pbp_categorical_columns].astype("category")
pbp_list.head()

In [ ]:
bspt_categorical_columns = [
    "gameId",
    "teamId",
    "personId",
    "firstName",
    "familyName",
    "nameI",
    "position",
]
boxscore_pt_list[bspt_categorical_columns] = boxscore_pt_list[
    bspt_categorical_columns
].astype("category")
boxscore_pt_list.head()

### Clock

We'll transform the clock data from a string into the total number of seconds.

In [ ]:
if pbp_list["clock"].dtype != "int64":
    pbp_list["clock"] = pbp_list["clock"].astype(str)
    pbp_list["minutes"] = pbp_list["clock"].str[2:4].astype(int)
    pbp_list["seconds"] = pbp_list["clock"].str[5:7].astype(int)
    pbp_list["clock"] = pbp_list["minutes"] * 60 + pbp_list["seconds"]
    pbp_list.drop(columns=["minutes", "seconds"], inplace=True)
pbp_list["clock"].head()

In [ ]:
if boxscore_pt_list["minutes"].dtype != "int64":
    boxscore_pt_list["minutes"] = boxscore_pt_list["minutes"].astype(str)
    boxscore_pt_list["mins"] = boxscore_pt_list["minutes"].str[:-3].astype(int)
    boxscore_pt_list["seconds"] = boxscore_pt_list["minutes"].str[-2:].astype(int)
    boxscore_pt_list["minutes"] = (
        boxscore_pt_list["mins"] * 60 + boxscore_pt_list["seconds"]
    )
    boxscore_pt_list.drop(columns=["mins", "seconds"], inplace=True)
boxscore_pt_list["minutes"].head()

In [ ]:
games_list.head()